In [21]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('../data/1.train_data.csv')
df = df.fillna({'location': 'nan'})
df.head()

,id,flg,keyword,location,text
0,20587,0,覚醒剤,icon:大トロ(@sato_wall_aim ),【マイリスト】カカカカ覚醒剤 https://t.co/15ZY9Oa2QS #sm3630...
1,31857,0,雷 落雷,ジャングル,@kenjikenji3343 せやな 落雷撃たれたら全員張り付くしかなかったからなぁ なん...
2,36793,0,血液,↓↓↓↓↓こちらをクリック↓↓↓↓,【カフェインの力】 体を動かす前にカフェインを摂ると交感神経が刺激され、体内の脂肪が分解され...
3,42299,0,ハリケーン,Close to your heart,あなたの鼓膜に直に響かせたい ハリケーンのように激しく愛したい 余計な壁なんて取っ払って 奥...
4,51449,0,殺人鬼,偽物なので悪しからず,@Mr8yt 違う違う。お化けは現実にいないけど殺人鬼は現実にいるじゃん〜。うーん、ホラーっ...


In [3]:
df[df.location == 'nan'].flg.value_counts()

0    7921
1     545
Name: flg, dtype: int64

In [4]:
df[df.location != 'nan'].flg.value_counts()

0    10930
1      889
Name: flg, dtype: int64

In [5]:
df_has_location = df[df.location != 'nan']
df_has_location.head()

,id,flg,keyword,location,text
0,20587,0,覚醒剤,icon:大トロ(@sato_wall_aim ),【マイリスト】カカカカ覚醒剤 https://t.co/15ZY9Oa2QS #sm3630...
1,31857,0,雷 落雷,ジャングル,@kenjikenji3343 せやな 落雷撃たれたら全員張り付くしかなかったからなぁ なん...
2,36793,0,血液,↓↓↓↓↓こちらをクリック↓↓↓↓,【カフェインの力】 体を動かす前にカフェインを摂ると交感神経が刺激され、体内の脂肪が分解され...
3,42299,0,ハリケーン,Close to your heart,あなたの鼓膜に直に響かせたい ハリケーンのように激しく愛したい 余計な壁なんて取っ払って 奥...
4,51449,0,殺人鬼,偽物なので悪しからず,@Mr8yt 違う違う。お化けは現実にいないけど殺人鬼は現実にいるじゃん〜。うーん、ホラーっ...


In [24]:
locations = df_has_location.location.to_list()
df_has_location.location.head(10)

0     icon:大トロ(@sato_wall_aim )
1                         ジャングル
2             ↓↓↓↓↓こちらをクリック↓↓↓↓
3           Close to your heart
4                    偽物なので悪しからず
5                            東京
6                          葺合けi
8                       教科書のすきま
10                           大坂
14                       海の無い東海
Name: location, dtype: object

## 都道府県名をlocationに含むか含まないか
- 含む場合：含まない場合=1:4
- 含む場合は15%くらいで正例
- 含まない場合は8%くらい

In [20]:
with open('../data/japanese_prefectures.txt') as f:
    prefectures = [line[:-1] for line in f.readlines()]
def in_prefecture_name(location: str):
    for p in prefectures:
        if p in location:
            return True
    return False
df_has_location.loc[:, 'in_prefecture'] = df_has_location.location.apply(in_prefecture_name)
df_has_location[df_has_location['in_prefecture']].flg.value_counts()

/Users/rikeda/Development/DisasterPrediction/env/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


0    2564
1     377
Name: flg, dtype: int64

In [18]:
df_has_location[~df_has_location['in_prefecture']].flg.value_counts()

0    8366
1     512
Name: flg, dtype: int64

## 地方名を含むかどうか
- 含まないものがほとんど．含まない方が負例っぽいので，相関はなさそう

In [25]:
with open('../data/japanese_regions.txt') as f:
    regions = [line[:-1] for line in f.readlines()]
def in_region_name(location: str):
    for p in regions:
        if p in location:
            return True
    return False
df_has_location.loc[:, 'in_region'] = df_has_location.location.apply(in_region_name)
df_has_location[df_has_location['in_region']].flg.value_counts()

0    195
1     17
Name: flg, dtype: int64

In [26]:
df_has_location[~df_has_location['in_region']].flg.value_counts()

0    10735
1      872
Name: flg, dtype: int64

## 県庁所在地名を含むかどうか
- 含むものはlocationが書かれている中で1/10くらい．含むものの方が正例である可能性が高いか

In [27]:
with open('../data/japanese_capitals.txt') as f:
    capitals = [line[:-1] for line in f.readlines()]
def in_capital_name(location: str):
    for p in capitals:
        if p in location:
            return True
    return False
df_has_location.loc[:, 'in_capital'] = df_has_location.location.apply(in_capital_name)
df_has_location[df_has_location['in_capital']].flg.value_counts()

0    1602
1     252
Name: flg, dtype: int64

In [28]:
df_has_location[~df_has_location['in_capital']].flg.value_counts()

0    9328
1     637
Name: flg, dtype: int64

## 英語名を含むかどうか
- 含まない方が若干正例である確率が高そうだけど，そこまで相関はなさそう

In [22]:
def in_eng(location: str):
    return len(re.findall(r'[A-Za-z]', location)) > 0

df_has_location.loc[:, 'in_eng'] = df_has_location.location.apply(in_eng)
df_has_location[df_has_location['in_eng']].flg.value_counts()

/Users/rikeda/Development/DisasterPrediction/env/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


0    1802
1     246
Name: flg, dtype: int64

In [23]:
df_has_location[~df_has_location['in_eng']].flg.value_counts()

0    9128
1     643
Name: flg, dtype: int64

### ローマ字で県名を含むかどうか，ローマ字で県庁所在地名を含むかどうか
- 5%もないくらいしか含まれていない
- ただし，県名が含まれていたら15%くらい，県庁所在地名が含まれていたら33%くらいで正例

In [33]:
with open('../data/roma_prefectures.txt') as f:
    roma_prefectures = [line[:-1] for line in f.readlines()]
def in_roma_prefecture_name(location: str):
    l = location.lower()
    for p in roma_prefectures:
        if p in l:
            return True
    return False
df_has_location.loc[:, 'in_roma_prefecture'] = df_has_location.location.apply(in_roma_prefecture_name)
df_has_location[df_has_location['in_roma_prefecture']].flg.value_counts()

0    328
1     64
Name: flg, dtype: int64

In [34]:
df_has_location[~df_has_location['in_roma_prefecture']].flg.value_counts()

0    10602
1      825
Name: flg, dtype: int64

In [35]:
with open('../data/roma_capitals.txt') as f:
    roma_capitals = [line[:-1] for line in f.readlines()]
def in_roma_capital_name(location: str):
    l = location.lower()
    for p in roma_capitals:
        if p in l:
            return True
    return False
df_has_location.loc[:, 'in_roma_capital'] = df_has_location.location.apply(in_roma_capital_name)
df_has_location[df_has_location['in_roma_capital']].flg.value_counts()

0    165
1     88
Name: flg, dtype: int64

In [36]:
df_has_location[~df_has_location['in_roma_capital']].flg.value_counts()

0    10765
1      801
Name: flg, dtype: int64

## その他，地名の手がかり語を含む
- 20%くらい含んでいる．
- 20%のうち，15%くらいが正例

In [43]:
place_clues = ['都', '市', '県', '市', '町', '区', '府', '村', '都市', '地方', '地域', '圏', '地区']
def in_place_clues(location: str):
    for p in place_clues:
        if p in location:
            return True
    return False
df_has_location.loc[:, 'in_place_clue'] = df_has_location.location.apply(in_place_clues)
df_has_location[df_has_location['in_place_clue']].flg.value_counts()

0    2160
1     331
Name: flg, dtype: int64

In [44]:
df_has_location[~df_has_location['in_place_clue']].flg.value_counts()

0    8770
1     558
Name: flg, dtype: int64

## 日本であることがわかる単語
- 10%くらい含んでいる．
- 10%のうち，12%くらいが正例

In [50]:
japan_clues = ['日本', 'japan']
def in_japan(location: str):
    l = location.lower()
    for p in japan_clues:
        if p in l:
            return True
    return False
df_has_location.loc[:, 'in_japan'] = df_has_location.location.apply(in_japan)
df_has_location[df_has_location['in_japan']].flg.value_counts()

0    1318
1     192
Name: flg, dtype: int64

In [46]:
df_has_location[~df_has_location['in_japan']].flg.value_counts()

0    9612
1     697
Name: flg, dtype: int64

## パターンマッチの組み合わせ

In [49]:
print(' === prefecture + clue === \n', df_has_location[df_has_location['in_prefecture'] & df_has_location['in_place_clue']].flg.value_counts())
print(' === region + clue === \n', df_has_location[df_has_location['in_region'] & df_has_location['in_place_clue']].flg.value_counts())
print(' === capital + clue === \n', df_has_location[df_has_location['in_capital'] & df_has_location['in_place_clue']].flg.value_counts())
print(' === prefecture + capital + clue === \n', df_has_location[df_has_location['in_prefecture'] & df_has_location['in_capital'] & df_has_location['in_place_clue']].flg.value_counts())

 === prefecture + clue === 
 0    1508
1     286
Name: flg, dtype: int64
 === region + clue === 
 0    62
1     8
Name: flg, dtype: int64
 === capital + clue === 
 0    1072
1     203
Name: flg, dtype: int64
 === prefecture + capital + clue === 
 0    985
1    190
Name: flg, dtype: int64
